# Regression Analysis on Student Grades

In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy as sql

In [47]:
# Import CSV file into a DataFrame; semicolons are the delineator in this file
df = pd.read_csv('data.csv', sep=';')

# Check for null or duplicate entries
print('Total null entries:', df.isnull().sum().sum())
print('Duplicate rows:', df.duplicated().value_counts())

# # If found:
# df = df.dropna().reset_index(drop=True)
# df = df.drop_duplicates(inplace=True)

print(type(df.duplicated()))
print(df.shape[0])


Total null entries: 0
Duplicate rows: False    316
Name: count, dtype: int64
<class 'pandas.core.series.Series'>
316


## EDA

In [ ]:
# Show info about each column: order, name, count, nulls, datatype
print(df.info())

# Display stats of numeric columns - count, mean, std, etc.
print(df.describe())

In [ ]:
# Try one-hotting every nominal column. Should total 58 dimensions

# Generate one-hot columns of every "object" column (nominal values)
df_dummies = pd.get_dummies(df.select_dtypes(include='object'))

# Join new columns to dataset
df = pd.concat([df, df_dummies], axis=1)

# Drop original "object" columns
df = df.drop(df.select_dtypes(include='object').columns, axis=1)

In [51]:
print(df.shape)
df.info()

(316, 59)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 59 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   age                316 non-null    int64
 1   Medu               316 non-null    int64
 2   Fedu               316 non-null    int64
 3   traveltime         316 non-null    int64
 4   studytime          316 non-null    int64
 5   failures           316 non-null    int64
 6   famrel             316 non-null    int64
 7   freetime           316 non-null    int64
 8   goout              316 non-null    int64
 9   Dalc               316 non-null    int64
 10  Walc               316 non-null    int64
 11  health             316 non-null    int64
 12  absences           316 non-null    int64
 13  G1                 316 non-null    int64
 14  G2                 316 non-null    int64
 15  G3                 316 non-null    int64
 16  school_GP          316 non-null    bool 
 17  school

In [72]:
corr_matrix = df.corr()
# plt.figure(figsize=(20,20))
# sns.heatmap(corr_matrix, cmap='coolwarm', annot=True, fmt=".2f") 
# plt.show()
# corr_matrix

result = corr_matrix[(corr_matrix.abs() > 0.5) & (corr_matrix.abs() < 1)].stack()

result_df = result.reset_index()
result_df.columns = ['Row', 'Column', 'Value']

result_df

,Row,Column,Value
0,Medu,Fedu,0.610685
1,Fedu,Medu,0.610685
2,Dalc,Walc,0.649514
3,Walc,Dalc,0.649514
4,G1,G2,0.838290
5,G1,G3,0.788582
6,G2,G1,0.838290
7,G2,G3,0.895996
8,G3,G1,0.788582
9,G3,G2,0.895996
